# Improving calculation of multivariate multiple regression

A small prototype for multivariate multiple regression computation assuming fixed residual covariance matrix.

In [64]:
simulate_multivariate = function(n=1000,p=7000,r=50) {
  set.seed(1)
  X = matrix(rnorm(n*p,3,4),n,p)
  beta = matrix(runif(p*r)>0.5, p, r)
  y = X %*% beta + do.call(cbind, lapply(1:r, function(i) rnorm(n)))
  X = susieR:::safe_colScale(X)
  y = y - apply(y,2,mean)
  scaled_prior_variance = 0.2
  L = 10
  return(list(X=X,y=y, d=diag(t(X)%*%X), n=n,p=p,r=r,V=scaled_prior_variance * cov(y),b=beta,L=L))
}
data = simulate_multivariate()

Here we do not use an existing R routine because `lm` is very slow (even using `.lm.fit` compared to providing pre-computed `d` as done below). 



In [65]:
method_1 = function(d,X,y,residual_covar,p) {
    XtY=t(X)%*%y
    bhat = diag(1/d) %*% XtY
    s2 = diag(residual_covar)
    sbhat = sqrt(do.call(rbind, lapply(1:p, function(j) s2 / d[j])))
    return(list(b=bhat,s=sbhat))
}

In [66]:
method = method_1

In [67]:
res = with(data, method(d,X,y,diag(r),p))

In [ ]:
The speed is decent but the code seems ugly. Must be a more elegant matrix representation (and hopefully faster)?